In [31]:
# Uploading "BreastTissue.xls"

# from google.colab import files
# uploaded = files.upload()


In [32]:
! apt-get install default-jre
!java -version
! pip install h2o

# Load the H2O library and start up the H2O cluter locally
import h2o
from h2o.automl import H2OAutoML

import pandas as pd
h2o.init(nthreads = -1, max_mem_size = 4)

readFile = pd.read_excel (uploaded["BreastTissue.xls"], 'Data')
readFile.to_csv ("BreastTissue.csv", index = None, header=True)

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("BreastTissue.csv")

x = train.columns
y="Class"
x.remove(y)
# Removing case index which doesnt influence data
x.remove("Case #")
# Run AutoML (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=50, seed=1)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,53 mins 15 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.6
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_863ke4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.999 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [33]:
aml.train(x=x, y=y, training_frame=train)
# View the AutoML Leaderboard
lb = aml.leaderboard
print("leader")
print(lb)

AutoML progress: |████████████
12:56:07.82: Skipping training of model GBM_5_AutoML_20200712_125521 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200712_125521.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 106.0.


████████████████████████████████████████████| 100%
leader


model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_grid__1_AutoML_20200712_125521_model_3,0.253307,1.69341,0.484863,0.235092
XGBoost_grid__1_AutoML_20200712_125521_model_12,0.25999,0.788367,0.518195,0.268527
XGBoost_1_AutoML_20200712_125521,0.25999,0.821389,0.523756,0.27432
XGBoost_grid__1_AutoML_20200712_125521_model_3,0.270043,1.11289,0.656244,0.430656
DeepLearning_grid__3_AutoML_20200712_125521_model_4,0.271341,3.1305,0.524244,0.274832
GBM_grid__1_AutoML_20200712_125521_model_4,0.274639,0.857701,0.482326,0.232638
DeepLearning_grid__2_AutoML_20200712_125521_model_4,0.275237,2.30591,0.504,0.254016
XGBoost_grid__1_AutoML_20200712_125521_model_7,0.275334,0.785947,0.501346,0.251348
StackedEnsemble_AllModels_AutoML_20200712_125521,0.285122,0.978002,0.567106,0.321609
GBM_1_AutoML_20200712_125521,0.287602,1.02532,0.503901,0.253916


In [34]:

model_id=aml.leader.params["model_id"]["actual"]["name"]
h2o.get_model(model_id).download_mojo()


'/content/DeepLearning_grid__1_AutoML_20200712_125521_model_3.zip'

In [35]:
aml.leader.confusion_matrix(train)


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,adi,car,con,fad,gla,mas,Error,Rate
0,22.0,0.0,0.0,0.0,0.0,0.0,0.000000,0 / 22
1,0.0,19.0,0.0,1.0,0.0,1.0,0.095238,2 / 21
2,0.0,0.0,14.0,0.0,0.0,0.0,0.000000,0 / 14
3,0.0,1.0,0.0,12.0,2.0,0.0,0.200000,3 / 15
4,0.0,0.0,0.0,0.0,15.0,1.0,0.062500,1 / 16
5,0.0,2.0,0.0,1.0,3.0,12.0,0.333333,6 / 18
6,22.0,22.0,14.0,14.0,20.0,14.0,0.113208,12 / 106
